<a href="https://colab.research.google.com/github/gbessardon/LAI_treatement/blob/main/Create_ECO-SG_chunk/Create_ecosg_tif_chunk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# if running in google colab

In [1]:
colab=1
if colab==1:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


# Declare libraries

In [2]:
import gdal
from osgeo import gdalconst
import numpy as np
import os

# Declare input/output filenames

In [3]:
ECOSGinput='/content/drive/MyDrive/ECOCLIMAP_SG/ecosg_final_map.dir'
MODISinput='/content/drive/MyDrive/MODIS_bolli/modis_lai_mym_v2_2012-2021_07.28.tif'# MODIS data
LAIinput='/content/drive/MyDrive/LAI_treatment_exp/LAI_0725_c.dir_2'
output_LAI='/content/drive/MyDrive/MODIS_bolli/LAI_0725_c.dir_2'

# Declare function

## Function to extract modis file extent

In [4]:
def extractmodisextent(fnmodis):
  jsoninfo=gdal.Info(fnmodis,format='json')
  EXTENT=jsoninfo.get('wgs84Extent')
  xmax,ymax=EXTENT.get('coordinates')[0][0]
  xmin,ymin=EXTENT.get('coordinates')[0][1]
  return(xmax,ymax,xmin,ymin)

In [5]:
def extractECOSGextent(fnecosg):
  ds=gdal.Open(fnecosg)
  gt=ds.GetGeoTransform()
  xres = gt[1]
  yres = gt[5]
#
  xmin = gt[0]
  ymax = gt[3]
#
  xmax = gt[0] + (xres * ds.RasterXSize)
  ymin = gt[3] + (yres * ds.RasterYSize)
  return(xmax,ymax,xmin,ymin)

## Function to cut ECOSG betwen latmin,latmax,lonmin,lonmax

In [6]:
def cutSG(fnSG,outfn,latmin=79,latmax=89.99,
          lonmin=-180,lonmax=90,proj='EPSG:4326'):
  src_fn=fnSG
  ds=gdal.Open(src_fn)
  gt=ds.GetGeoTransform()
  gdal.Warp(outfn,ds,
            outputBounds =(lonmin,latmin,lonmax,latmax), 
            xRes=gt[1], yRes=gt[5],targetAlignedPixels=True,
            dstSRS=proj)

## Function to reproject MODISfile following the ECOSG cutted file

In [7]:
 def reproject_modis(MODISfile,fnecosg,dst_filename):  
    
    src=gdal.Open(MODISfile)
    src_proj = src.GetProjection()
    src_geotrans = src.GetGeoTransform()

    #Open the ecosg file to get the projection details
    match_filename = fnecosg
    match_ds = gdal.Open(match_filename)
    match_proj = match_ds.GetProjection()
    match_geotrans = match_ds.GetGeoTransform()
    wide = match_ds.RasterXSize
    high = match_ds.RasterYSize

    #Create the MODIS tiff file with the right projection
    dst = gdal.GetDriverByName('GTiff').Create(dst_filename, wide, high, 1, gdal.GDT_Byte)
    dst.SetGeoTransform( match_geotrans )
    dst.SetProjection( match_proj)
    gdal.ReprojectImage(src, dst, src_proj, match_proj, gdalconst.GRA_Bilinear)

    del dst
    return

# Function to merge all data together

In [8]:
def mergetogether(INPUT_LIST,outputfiles):
  ds=gdal.Open(INPUT_LIST[0])
  driver=ds.GetDriver()
  gt=ds.GetGeoTransform()
  xres = gt[1]
  yres = gt[5]
  xmin = gt[0]
  ymax = gt[3]
#
  xmax = gt[0] + (xres * ds.RasterXSize)
  ymin = gt[3] + (yres * ds.RasterYSize)
  w=ds.RasterXSize
  h=ds.RasterYSize
  dsv=gdal.BuildVRT('test.vrt',INPUT_FILES,
                    outputBounds=(xmin, ymin, xmax, ymax), 
                    xRes=xres, yRes=yres, targetAlignedPixels=True)
  opt=gdal.WarpOptions(format=driver.ShortName,
                  outputBounds=(xmin, ymin, xmax, ymax),
                  width=w,height=h)
  gdal.Warp(outputfiles,'test.vrt',options=opt)
  os.remove('test.vrt')

# Main

In [9]:
ECOSGoutput='temp.tif'
MODISoutput='modis_repojected.tif'
INPUT_FILES=[ECOSGinput,LAIinput,MODISoutput]

# Get the intersection between MODIS and ECOSG

## Extract modis file extent

In [10]:
(xmax,ymax,xmin,ymin)=extractmodisextent(MODISinput)

## Extract ecosg file extent

In [11]:
(xmaxsg,ymaxsg,xminsg,yminsg)=extractECOSGextent(ECOSGinput)

# Get the intersction limits (avoids issue with extra points)

In [12]:
XMIN=max(xmin,xminsg)
YMIN=max(ymin,yminsg)
XMAX=min(xmax,xmaxsg)
YMAX=min(ymax,ymaxsg)


## Cut ECO-SG between modis extent

In [13]:
cutSG(ECOSGinput,ECOSGoutput,latmin=YMIN,latmax=YMAX,
          lonmin=XMIN,lonmax=XMAX)

# Reproject modis

In [14]:
reproject_modis(MODISinput,ECOSGoutput,MODISoutput)

# Merge all the data together

In [ ]:

ds=gdal.Open(INPUT_FILES[0])
driver=ds.GetDriver()
gt=ds.GetGeoTransform()
xres = gt[1]
yres = gt[5]
xmin = gt[0]
ymax = gt[3]
#
xmax = gt[0] + (xres * ds.RasterXSize)
ymin = gt[3] + (yres * ds.RasterYSize)
w=ds.RasterXSize
h=ds.RasterYSize
dsv=gdal.BuildVRT('test.vrt',INPUT_FILES,
                  outputBounds=(xmin, ymin, xmax, ymax))
dsv.FlushCache()
opt=gdal.WarpOptions(format=driver.ShortName,
                  outputBounds=(xmin, ymin, xmax, ymax),
                  width=w,height=h)
gdal.Warp(output_LAI,dsv,options=opt)
#os.remove('test.vrt')

['/content/drive/MyDrive/ECOCLIMAP_SG/ecosg_final_map.dir',
 '/content/drive/MyDrive/LAI_treatment_exp/LAI_0725_c.dir_2',
 'modis_repojected.tif']

In [15]:
mergetogether(INPUT_FILES,output_LAI)
os.remove(ECOSGoutput)
os.remove(MODISoutput)

SystemError: ignored